In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
import os
import pandas as pd
from os.path import exists
from pathlib import Path
import logging
COBRAKBase_path = "/Users/chenry/code/cobrakbase"
BaseModule_path = "/Users/chenry/code/KBBaseModules"
ModelSEEDpy_path = "/Users/chenry/code/ModelSEEDpy"
ModelSEEDReconstruction_path = "/Users/chenry/code/KB-ModelSEEDReconstruction"
sys.path.append(COBRAKBase_path)
sys.path.append(BaseModule_path)
sys.path.append(ModelSEEDpy_path)
sys.path.append(ModelSEEDReconstruction_path+"/lib")
from modelseedpy import MSPackageManager, MSModelUtil, MSBuilder, MSATPCorrection, MSGapfill
from modelseedpy.core.mstemplate import MSTemplateBuilder
from ModelSEEDReconstruction.modelseedrecon import ModelSEEDRecon
from installed_clients.WorkspaceClient import Workspace
#You need to get a KBase dev token or pull a token from the narrative and put in ~/.kbase/token file
token = None
if Path(str(Path.home()) + '/.kbase/token').exists():
    with open(str(Path.home()) + '/.kbase/token', 'r') as fh:
        token = fh.read().strip()
workspace_url = "https://appdev.kbase.us/services/ws"#For appdev
#workspace_url = "https://kbase.us/services/ws"#For prod

python version 3.9.13
cobrakbase 0.3.1


In [2]:
wsclient = Workspace(workspace_url, token=token)
msrecon = ModelSEEDRecon({"ATP_media_workspace":"68393","version":"0.0.1"},ModelSEEDReconstruction_path,ModelSEEDReconstruction_path+"/notebooks/",token,{"Workspace":wsclient},None)

In [3]:
genome = msrecon.get_msgenome("68303/Escherichia_coli_K-12_MG1655.RAST")
core_template = msrecon.get_template("core_template_sulfur3","NewKBaseModelTemplates")
template = msrecon.get_template("GramNegModelTemplateV4","NewKBaseModelTemplates")

In [4]:
mdl = MSBuilder(genome, template).build("Escherichia_coli_K-12_MG1655.RAST.mdl", '0', False, False)
mdl.genome = genome#Move into MSBuilder?
mdl.template = template#Move into MSBuilder?
mdlutl = MSModelUtil.get(mdl)
atpcorrection = MSATPCorrection(mdlutl,core_template,[],load_default_medias=True,max_gapfilling=10,gapfilling_delta=0,forced_media=[],default_media_path=msrecon.module_dir+"/data/atp_medias.tsv")
tests = atpcorrection.run_atp_correction()
for test in tests:
    print(test["media"].id+":"+str(test["threshold"]))
print("Gapfilling:",len(atpcorrection.cumulative_core_gapfilling))

Genome custom GC: 0.508


INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd08701 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd08701 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
DEBUG:modelseedpy.core.msmodelutl:Expansion started! Binary = True
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f9008caea00>, 'is_max_threshold': True, 'threshold': 1.2000000000000002, 'objective': 'rxn00062_c0'}
DEBUG:modelseedpy.core.msmodelutl:Expansion time:Glc:4.781599327999999
DEBUG:modelseedpy.core.msmodelut

Removing rxn10157_c0 <
Removing rxn09295_c0 <
Removing rxn05938_c0 <
Removing rxn08628_c0 >
Removing rxn10155_c0 <
Removing rxn01353_c0 <
Removing rxn05683_c0 <
Removing rxn09193_c0 <
Removing rxn09003_c0 <
Removing rxn01128_c0 >
Removing rxn08655_c0 <
Removing rxn09272_c0 <
Removing rxn05313_c0 <
Removing rxn01510_c0 >
Removing rxn05297_c0 >
Removing rxn00507_c0 <
Removing rxn05596_c0 <
Removing rxn01674_c0 <
Removing rxn01679_c0 <
Removing rxn00778_c0 >
Removing rxn05206_c0 >
Removing rxn00239_c0 <
Removing rxn05937_c0 <
Removing rxn00715_c0 <
Removing rxn05638_c0 >
Removing rxn05289_c0 >
Removing rxn00839_c0 <
Removing rxn14159_c0 <
Removing rxn08233_c0 >
Removing rxn05654_c0 <
Removing rxn02914_c0 <
Removing rxn00265_c0 <
Glc:1.2000000000000042
Pyr:0.5999999999999996
Fum:0.29999999999999927
For/NO2:0.8000000000000033
For/NO3:3.9000000000000057
Pyr/NO3:7.20000000000015
Ac/NO3:4.349999999999983
Glc/DMSO:1.1999999999999993
Glc/TMAO:1.1999999999999993
Pyr/DMSO:0.6000000000000015
Pyr/TM

In [5]:
msrecon.save_model(mdlutl,68304)

In [6]:
media = msrecon.kbase_api.get_from_ws("KBaseMedia/Carbon-D-Glucose",None)
tests = mdlutl.atputl.build_tests()
msgapfill = MSGapfill(mdlutl,[template],[],tests,blacklist=[])
mdlutl.gfutl.cumulative_gapfilling = []
gfresults = msgapfill.run_gapfilling(media,"bio1",0.01)
msgapfill.integrate_gapfill_solution(gfresults,mdlutl.gfutl.cumulative_gapfilling)
mdlutl.model.objective = "bio1"
mdlutl.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print(media.id+":"+str(mdlutl.model.slim_optimize()))

modelseedpy.fbapkg.gapfillingpkg - CRITICAL - Model cannot achieve the minimum objective even before filtering!
CRITICAL:modelseedpy.fbapkg.gapfillingpkg:Model cannot achieve the minimum objective even before filtering!
DEBUG:modelseedpy.core.msmodelutl:Expansion started! Binary = True
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f9008caea00>, 'is_max_threshold': True, 'threshold': 1.2000000000000002, 'objective': 'rxn00062_c0'}


Objective before filtering: 0.0 ; min objective: 0.01


KeyboardInterrupt: 

In [ ]:
msrecon.save_model(mdlutl,params["workspace"])
mdlutl.model.objective = "bio1"
mdlutl.pkgmgr.getpkg("KBaseMediaPkg").build_package(None)
print("Complete:"+str(mdlutl.model.slim_optimize()))

In [3]:
output = msrecon.build_metabolic_models({
    "workspace":68304,
    "genome_refs":["68303/Escherichia_coli_K-12_MG1655.RAST"],
    "run_gapfilling":True,
    "atp_safe":True,
    "forced_atp_list":[],
    "gapfilling_media_list":["KBaseMedia/Carbon-D-Glucose"],
    "suffix":".mdl",
    "core_template":"auto",
    "gs_template":"auto",
    "gs_template_ref":None,
    "template_reactions_only":True,
    "output_core_models":True
})

/Users/chenry/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318 UserWarning: Trying to unpickle estimator KNeighborsClassifier from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
INFO:modelseedpy.core.msmodelutl:cpd11632 not found in model!
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
INFO:modelseedpy.core.msmodelutl:cpd01024 not found in model!
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solver.py:416 UserWarning: solver status is 'infeasible'
cobra/util/solv

Removing rxn10157_c0 <
Removing rxn08628_c0 >
Removing rxn10155_c0 <
Removing rxn09193_c0 <
Removing rxn00704_c0 >
Removing rxn05297_c0 >
Removing rxn05596_c0 <
Removing rxn00778_c0 >
Removing rxn05206_c0 >
Removing rxn05638_c0 >
Removing rxn14159_c0 <
Removing rxn08233_c0 >
Removing rxn02201_c0 >
Removing rxn00351_c0 >
Removing rxn00836_c0 <
Removing rxn05318_c0 <
Removing rxn05318_c0 >
Removing rxn01094_c0 >
Removing rxn08709_c0 >
Removing rxn00423_c0 >
Removing rxn00364_c0 <
Removing rxn00364_c0 >
Removing rxn03408_c0 <
Removing rxn03408_c0 >
Removing rxn02177_c0 >
Removing rxn05250_c0 >
Removing rxn00646_c0 >
Removing rxn05440_c0 >
Removing rxn01673_c0 <
Removing rxn01673_c0 >
Removing rxn05156_c0 >
Removing rxn00642_c0 >
Removing rxn00022_c0 >
Removing rxn02000_c0 <
Removing rxn02000_c0 >
Removing rxn00322_c0 >
Removing rxn05457_c0 <
Removing rxn05457_c0 >
Removing rxn00107_c0 >
Removing rxn13783_c0 >
Removing rxn08934_c0 <
Removing rxn08934_c0 >
Removing rxn04600_c0 <
Removing rx

cobra/core/model.py:727 UserWarning: rxn09231_c0: cpd00008_c0 + cpd15481_c0 --> cpd00001_c0 + cpd00177_c0 + cpd15480_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn00800_c0: cpd02375_c0 --> cpd00018_c0 + cpd00106_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn08172_c0: cpd00006_c0 + cpd03282_c0 --> cpd00005_c0 + cpd00067_c0 + cpd02211_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn01705_c0: cpd00356_c0 + cpd00367_c0 --> cpd00046_c0 + cpd00067_c0 + cpd00533_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn00211_c0: cpd00001_c0 + 2 cpd00003_c0 + cpd00026_c0 --> 2 cpd00004_c0 + 3 cpd00067_c0 + cpd00144_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn05654_c0: cpd00036_e0 + cpd00067_e0 --> cpd00036_c0 + cpd00067_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarn


Removing rxn01218_c0 >
Removing rxn00509_c0 >
Removing rxn02898_c0 >
Removing rxn05654_c0 <
Removing rxn05654_c0 >
Removing rxn01199_c0 >
Removing rxn02319_c0 >
Removing rxn08807_c0 <
Removing rxn08807_c0 >
Removing rxn05614_c0 >
Removing rxn10344_c0 >
Removing rxn02320_c0 <
Removing rxn02320_c0 >
Removing rxn03384_c0 >
Removing rxn05167_c0 >
Removing rxn02504_c0 <
Removing rxn02504_c0 >
Removing rxn04082_c0 <
Removing rxn04082_c0 >
Removing rxn00262_c0 >
Removing rxn03173_c0 >
Removing rxn03039_c0 <
Removing rxn03039_c0 >
Removing rxn10262_c0 <
Removing rxn10262_c0 >
Removing rxn10209_c0 >
Removing rxn09313_c0 <
Removing rxn09313_c0 >
Removing rxn08084_c0 >
Removing rxn10225_c0 >
Removing rxn00898_c0 >
Removing rxn05444_c0 >
Removing rxn05334_c0 <
Removing rxn05334_c0 >
Removing rxn10205_c0 >
Removing rxn05235_c0 >
Removing rxn02090_c0 <
Removing rxn02090_c0 >
Removing rxn05377_c0 <
Removing rxn05377_c0 >
Removing rxn05582_c0 <
Removing rxn05582_c0 >
Removing rxn01843_c0 <
Removing r

cobra/core/model.py:727 UserWarning: rxn08137_c0: cpd00001_c0 + cpd02511_c0 --> cpd00002_c0 + cpd00008_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn05369_c0: cpd11505_c0 --> cpd00001_c0 + cpd11506_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn01322_c0: cpd00232_c0 --> cpd00020_c0 + cpd00492_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn08376_c0: cpd00067_e0 + cpd00412_e0 --> cpd00067_c0 + cpd00412_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn08895_c0: cpd00001_c0 + cpd15506_c0 --> cpd15505_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn08817_c0: cpd00001_c0 + cpd15350_c0 --> 2 cpd00067_c0 + cpd00080_c0 + cpd01741_c0 not in Escherichia_coli_K-12_MG1655.RAST.mdl
cobra/core/model.py:727 UserWarning: rxn00910_c0: cpd00006_c0 + cpd00345_c0 --> cpd00005_c0 + cpd00067_c0 + cpd00125_c0 

...

Objective after filtering: 0.0 ; min objective: 0.01


DEBUG:modelseedpy.core.msmodelutl:Expansion started!
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f94189eeb50>, 'is_max_threshold': True, 'threshold': 31.799999999999997, 'objective': 'rxn00062_c0'}


Reactions unfiltered: 2676


DEBUG:modelseedpy.core.msmodelutl:Expansion time:Glc/O2:0.8747417859999587
DEBUG:modelseedpy.core.msmodelutl:Filtered count:1 out of 253
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f94189eeb90>, 'is_max_threshold': True, 'threshold': 9.0, 'objective': 'rxn00062_c0'}
DEBUG:modelseedpy.core.msmodelutl:Expansion time:Ac/O2:0.21443183699989277
DEBUG:modelseedpy.core.msmodelutl:Filtered count:1 out of 253
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f94189ee990>, 'is_max_threshold': True, 'threshold': 14.7, 'objective': 'rxn00062_c0'}
DEBUG:modelseedpy.core.msmodelutl:Expansion time:Etho/O2:0.21547689099998024
DEBUG:modelseedpy.core.msmodelutl:Filtered count:1 out of 253
DEBUG:modelseedpy.core.msmodelutl:testing condition {'media': <modelseedpy.core.msmedia.MSMedia object at 0x7f94189ee510>, 'is_max_threshold': True, 'threshold': 12.45, 'objective': 'rxn00062_c0'}

TypeError: 'NoneType' object is not subscriptable